In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

2022-01-25 19:59:13.808374: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/navneeth/ros2_ws/install/my_robot_interfaces/lib:/usr/lib/x86_64-linux-gnu/gazebo-11/plugins:/opt/ros/foxy/opt/yaml_cpp_vendor/lib:/opt/ros/foxy/opt/rviz_ogre_vendor/lib:/opt/ros/foxy/lib/x86_64-linux-gnu:/opt/ros/foxy/lib
2022-01-25 19:59:13.808402: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [18]:
def log_loss(y_true,y_predicted):
    eplison = 1e-15
    y_predicted_new = [max(i,eplison) for i in y_predicted] #making zeros close to zero
    y_predicted_new = [min(i,1- eplison) for i in y_predicted_new] #making ones close to ones
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new) + (1-y_true)*np.log(1 - y_predicted_new))

In [19]:
def sigmoid(X):
    return 1 / (1+ np.exp(-X))

In [31]:
class myNN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0
        
    def fit(self,X,y,epochs,loss_thresold):
        self.w1 , self.w2, self.bias  = self.gradient_descent(X['age'],X['affordibility'],y,epochs,loss_thresold)
        
    def predict(self,X_test):
        weighted_sum = self.w1 * X_test['age'] + self.w2 * X_test['affordibility'] + self.bias
        return sigmoid(weighted_sum)
        
    def gradient_descent(self , age,affordability,y_true,epochs, loss_thresold):
        w1=w2=1
        bias = 0
        rate = 0.5
        n=len(age)

        for i in range(epochs):
            weighted_sum = w1*age + w2*affordability + bias
            y_predicted = sigmoid(weighted_sum)

            loss = log_loss(y_true,y_predicted)

            dw1 = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
            dw2 = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true))
            bias_d = np.mean(y_predicted-y_true)
            w1 = w1 - rate*dw1
            w2 = w2 - rate*dw2
            bias = bias - rate*bias_d
            print(f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
            if loss<=loss_thresold :
                break

        return w1 , w2 , bias


In [32]:
df = pd.read_csv('./datasets/insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2, random_state=25)


In [34]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100


In [35]:
customModel = myNN()
customModel.fit(X_train_scaled,y_train,epochs=5,loss_thresold=0.4631)

Epoch:0, w1:0.974907633470177, w2:0.948348125394529, bias:-0.11341867736368583, loss:0.7113403233723417
Epoch:1, w1:0.9556229728273669, w2:0.9058873696677865, bias:-0.2122349122718517, loss:0.6812647787377568
Epoch:2, w1:0.9416488476693794, w2:0.8719790823960313, bias:-0.29775789977965383, loss:0.6591474252715025
Epoch:3, w1:0.9323916996249162, w2:0.8457541517722915, bias:-0.37150947240035115, loss:0.6431523291301916
Epoch:4, w1:0.9272267472726993, w2:0.8262362885332687, bias:-0.4350664302689159, loss:0.6316873063379158


In [36]:
customModel.predict(X_test_scaled)

2     0.695713
10    0.636006
21    0.451655
11    0.657189
14    0.699625
9     0.722476
dtype: float64

In [37]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64